Импортируем все необходимые библиотеки

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

/home/d.milovanov/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/d.milovanov/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/d.milovanov/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/d.milovanov/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/d.milovanov/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in

Загружаем датасет из csv файла и смотрим на первые пять записей

In [4]:
df = pd.read_csv('/home/d.milovanov/nasa_fires/forest.csv')
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


Теперь посмотрим на характеристики целевой переменной. Первое, что мы видим, что целевая переменная представлена типом float64, и это значит, что мы столкнулись с задачей регрессии. Второе мы видим, что в данных есть экстремально высокие площади возгораний (целевая переменная). Есть разные способы борьбы с outliers(выбросами), заменяют на 0, на медиану или среднее, выбрасывают из датасета. Но тут поступим иначе, датасет и так маленький, а выбрасывая даже 1 наблюдение, теряем существенную часть информации, которая могла бы помочь в обучении модели, так что прологарифмируем целевую переменную.

In [6]:
df['area'].describe()

count     517.000000
mean       12.847292
std        63.655818
min         0.000000
25%         0.000000
50%         0.520000
75%         6.570000
max      1090.840000
Name: area, dtype: float64

Логарифмируем целевую переменную, так как есть нули применяем вот такой хак, сразу же создаем переменную, которую будем использовать для обучении моделей и предсказаний на кросс-валидации. 

In [4]:
x = np.ma.log(df.area.values)
y_train = x.filled(0)

Удаляем из датасета колонку с переменной, которую нам надо предсказывать в задаче регрессии, а также заменяем категориальные переменные при помощи подохода One-Hot Encoding.

In [5]:
X_train = df.drop(['area'], axis=1)
X_train = pd.get_dummies(df, columns=['month', 'day', 'X', 'Y'])

Функция для вычисления ошибки на 10 фолдовой кросс-валидации

In [6]:
def rms_cross_val(model):
    kfold = KFold(10, shuffle=True, random_state=85).get_n_splits(X_train.values)
    rmse = np.sqrt(-cross_val_score(model, X_train.values, y_train, scoring="neg_mean_squared_error", cv = kfold))
    return(rmse)

Будем использовать градиентный бустинг из sklearn, библиотеку Xgboost и LightGMB. Все модели с параметрами по умолчанию, можно немного подкрутить их гиперпараметры для получения меньшей ошибки, назначения гиперпараметров можно посмотреть в документации к библиотекам

In [7]:
Gradient_Boost = GradientBoostingRegressor()
xgb_model = xgb.XGBRegressor()
lgb_model = lgb.LGBMRegressor()

models = {
          'Gradient_boost_sklearn': Gradient_Boost, 
          'XGBoost': xgb_model, 
          'lightbm': lgb_model
         }

In [9]:
for name, model in models.items():
    score = rms_cross_val(model)
    print(f"{name} score: {score.mean()}, {score.std()}")

Gradient_boost_sklearn score: 0.05768924683828015, 0.0677040089134712
XGBoost score: 0.05904038655561926, 0.06712641665169697
lightbm score: 0.1986825050298882, 0.17741740084907595


Как видно наиболее низкую ошибку показала модель градиентного бустинга из библиотеки sklearn, еще можно было попробовать сделать ensemble модель для этих 3 моделей градиентного бустинга, то есть обьединить их предсказания